# EEG Feature Extraction 2: Parameterizing PSDs and Modeling
In 2008, a BCI Competition was held on EEG datasets to find the best ML and statistical algorithms to differentiate different classes of neural data. The BCI Competition IV 2b is a motor imagery dataset with eye artifact data, making it a very realistic dataset. The subjects are prompted to imagine left vs right hand movement and the EEG + EOG signals for each trial are collected. We here have provided a simpler version of the dataset in CSV format for you to get started with. 

This notebook will introduce a couple more feature extraction methods, provide some guidance in dimensionality reduction, and show how these features can be used in a model to predict classes. 

Terminology: <br>
- <b>Fitting Oscillations and One Over F (FOOOF)</b>: A technique that fits a 1/f model to the Power Spectrum Density to allow for more features to be extracted. 
- <b>Modeling and NN</b>: Machine learning techniques that can help us classify our data. 

Experiment Setup: http://www.bbci.de/competition/iv/desc_2b.pdf <br>
FOOOF: https://fooof-tools.github.io/fooof/index.html


## Imports

In [ ]:
from pathlib import Path # For making paths compatible on Windows and Macs

import pickle # For loading and creating pickle files
import pandas as pd # For working with DataFrames 
import numpy as np # For ease of array manipulation, stats, and some feature extraction
import matplotlib.pyplot as plt # For plotting pretty plots :) 
import scipy.signal as signal # For calculating PSDs and plotting spectrograms

from neurodsp.spectral import compute_spectrum # For smoothed PSD computation
from neurodsp.timefrequency import amp_by_time, freq_by_time, phase_by_time # For neurodsp features

## Constants

In [ ]:
eeg_fs = 250 # Data was recorded at 250 Hz
eeg_chans = ["C3", "Cz", "C4"] # 10-20 system 
eog_chans = ["EOG:ch01", "EOG:ch02", "EOG:ch03"] 
all_chans = eeg_chans + eog_chans
event_types = {0:"left", 1:"right"}

## Helper Functions

In [ ]:
# Get Frequencies and mean PSDs from EEG data - this yeilds smoother PSDs because it averages the PSDs made from sliding windows. 
def getMeanFreqPSD(eeg_data, fs=eeg_fs):
    freq_mean, psd_mean = compute_spectrum(eeg_data, fs, method='welch', avg_type='mean', nperseg=fs*2)
    return freq_mean, psd_mean

## Load Epoched Data

In [ ]:
# These + epoched_test.pkl are the epochs that will be used in accuracy evaluation
epoch_df_filename = Path("./data/epoched_train.pkl")
eeg_epoch_full_df = pd.read_pickle(epoch_df_filename)
eeg_epoch_full_df.head(2)

## Load Features from last time 

In [ ]:
# To save your feature_df from last time, feature_df.to_pickle("W1_feature_df.pkl") in your 01_Notebook
W1_feature_df_filename = Path("./W1_feature_df.pkl")
W1_feature_df = pd.read_pickle(W1_feature_df_filename)
W1_feature_df.head(2)

# Let's add a couple more features! 

## NeuroDSP Alpha Instantaneous Amplitude median
Perhaps this can help account for large outliers and noise in the alpha power. 

In [ ]:
alpha_range = (7, 12)
alpha_amps = {}
for i in range(0, len(eeg_epoch_full_df)): 
    for ch in eeg_chans: 
        amp = amp_by_time(eeg_epoch_full_df[ch][i][:], eeg_fs, alpha_range)
        key = ch + "_" + str(alpha_range) + "_inst_med"
        if key not in alpha_amps: 
            alpha_amps[key] = list()
        alpha_amps[key].append(np.nanmedian(amp))

alpha_med_df = pd.DataFrame(alpha_amps)
display(alpha_med_df.head(2))

feature_df = pd.concat([W1_feature_df, alpha_med_df], axis=1)
display(feature_df.head(2))


## FOOOF (i.e. Fitting Oscillations and One Over F) 
PSDs are highly analyzed in neural data, and FOOOF helps parameterize some qualities of these PSDs that can allow us to consider more features of our data. In particular, this package will take a PSD and try to fit a 1/f aperiodic line to represent the base "noise" and then fit more curves above the line to outline the periodic activity. This can be helpful in being more specific about the oscillatory behavior in the neural data. 

Donoghue T, Haller M, Peterson E, Varma P, Sebastian P, Gao R, Noto T, Lara AH, Wallis JD,
Knight RT, Shestyuk A, Voytek B. Parameterizing neural power spectra into periodic
and aperiodic components. Nature Neuroscience (in press)

They have great documentation and tutorials for fine-tuning the models to best parameterize your data on their website: https://fooof-tools.github.io/fooof/auto_tutorials/index.html 

### Let's first understand visually what the model does: 
Here we can see what the different features of the PSDs are identified. 

In [ ]:
# Import required code for visualizing example models
from fooof import FOOOF
from fooof.sim.gen import gen_power_spectrum
from fooof.sim.utils import set_random_seed
from fooof.plts.annotate import plot_annotated_model

# Set random seed, for consistency generating simulated data
set_random_seed(10)

# Simulate example power spectra
freqs1, powers1 = gen_power_spectrum([3, 40], [1, 1],
                                     [[10, 0.2, 1.25], [30, 0.15, 2]])

# Initialize power spectrum model objects and fit the power spectra
fm1 = FOOOF(min_peak_height=0.05, verbose=False)
fm1.fit(freqs1, powers1)

plot_annotated_model(fm1, annotate_aperiodic=True)


### Let's see what the model predicts of one EEG trace

In [ ]:
# Constants 
freq_range = [1, 40]

In [ ]:
# Initialize a FOOOF object
fm = FOOOF(peak_width_limits=[1, 8], max_n_peaks=6, min_peak_height=0.4)

# Get the PSD of our EEG Signal
sig = eeg_epoch_full_df['Cz'][0]
freq, psd = getMeanFreqPSD(sig)

fm.add_data(freq, psd, freq_range)

fm.fit()
fm.report()

### Let's see what the model detects from the averaged PSDs that we saw before

In [ ]:
# Get PSD averages for each channel for each event type (0=left or 1=right)
psd_averages_by_type = {}

for event_type in event_types.keys(): 
    psds_only_one_type={}
    freqs_only_one_type={}
    for i, row in eeg_epoch_full_df[eeg_epoch_full_df["event_type"] == event_type].iterrows(): 
        for ch in eeg_chans: 
            if ch not in psds_only_one_type: 
                psds_only_one_type[ch] = list()
                freqs_only_one_type[ch] = list()
            f, p = getMeanFreqPSD(row[ch])
            psds_only_one_type[ch].append(p)
            freqs_only_one_type[ch].append(f)
    avg_psds_one_type = {}
    for ch in eeg_chans:
        psds_only_one_type[ch] = np.array(psds_only_one_type[ch])
        avg_psds_one_type[ch] = np.mean(psds_only_one_type[ch], axis=0)
    psd_averages_by_type[event_type] = dict(avg_psds_one_type)
    

In [ ]:
# Visualize the parameters in these two classes of C4 activity
fm_left_C4 = FOOOF(peak_width_limits=[1, 8], max_n_peaks=6, min_peak_height=0.4)
fm_left_C4.add_data(freqs_only_one_type[eeg_chans[0]][0], psd_averages_by_type[0]['C4'], freq_range)
fm_left_C4.fit()
fm_left_C4.report()

fm_right_C4 = FOOOF(peak_width_limits=[1, 8], max_n_peaks=6, min_peak_height=0.4)
fm_right_C4.add_data(freqs_only_one_type[eeg_chans[0]][0], psd_averages_by_type[1]['C4'], freq_range)
fm_right_C4.fit()
fm_right_C4.report()

In [ ]:
# Calculate central freq, alpha power, and bandwidth for each channel and each trial
# This cell takes a few minutes to run (~8 mins on my computer). There are 3680 trials in the training data. 

# Initialize a fooof object
fm = FOOOF(peak_width_limits=[1, 8], max_n_peaks=6, min_peak_height=0.4)

# Some will not have alpha peaks, use these variables to keep track
num_with_alpha = 0
num_without_alpha = 0
fooof_parameters = {}
for i in range(len(eeg_epoch_full_df)):
    # Print the trial number every 100 to make sure we're making progress
    if i % 100 == 0 :
        print(i)
    for ch in eeg_chans:
        # Determine the key
        CF_key = ch + "_alpha_central_freq"
        PW_key = ch + "_alpha_power"
        BW_key = ch + "_alpha_band_width"
        if CF_key not in fooof_parameters: 
            fooof_parameters[CF_key] = []
        if PW_key not in fooof_parameters: 
            fooof_parameters[PW_key] = []
        if BW_key not in fooof_parameters: 
            fooof_parameters[BW_key] = []
        
        # Calculate the PSD for the desired signal
        sig = eeg_epoch_full_df[ch][i]
        freq, psd = getMeanFreqPSD(sig)
        
        # Set the frequency and spectral data into the FOOOF model and get peak params
        fm.add_data(freq, psd, freq_range)
        fm.fit()
        peak_params = fm.peak_params_
        
        # Only select the peaks within alpha power
        peak_params_alpha = []
        for param in peak_params: 
            if (param[0] > alpha_range[0]) and (param[0] < alpha_range[1]): 
                peak_params_alpha.append(param)
        
        # Take the average if there are multiple peaks detected, otherwise 0 everything
        means = []
        if len(peak_params_alpha) > 0: 
            num_with_alpha += 1
            means = np.mean(peak_params_alpha, axis=0)
        else :
            num_without_alpha += 1
            means = [0, 0, 0]
        
        fooof_parameters[CF_key].append(means[0])
        fooof_parameters[PW_key].append(means[1])
        fooof_parameters[BW_key].append(means[2])

In [ ]:
# Concatenate 
fooof_parameters_df = pd.DataFrame(fooof_parameters)
display(fooof_parameters_df.head(5))

feature_df = pd.concat([W1_feature_df, alpha_med_df, fooof_parameters_df], axis=1)
display(feature_df.head(2))
print("% with alpha:", num_with_alpha / (num_with_alpha + num_without_alpha))

In [ ]:
# Save it so you don't need to spend extra time rerunning the heavy computation cell from above. 
feature_df.to_pickle("W2_feature_df.pkl")

## Now you have many features to put into your Machine Learning algorithm or plot with statistics to find differences between the classes! 

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPooling2D, Flatten

In [ ]:
feature_df = pickle.load(open("W2_feature_df.pkl", "rb"))
feature_df.head()

In [ ]:
# Check for null 
null_df = pd.DataFrame(feature_df.isnull().mean(), columns=["null %"])
null_cols = null_df[null_df["null %"] > 0]
null_cols

In [ ]:
# View cleaned dataframe
cleaned_feature_df = feature_df.drop(list(null_cols.index), axis=1)
cleaned_feature_df.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(cleaned_feature_df.drop("y", axis=1), cleaned_feature_df["y"])
x_train.shape, x_test.shape

# Logistic Regression

In [ ]:
log_model = LogisticRegression()
log_model.fit(x_train, y_train)
train_preds = log_model.predict(x_train)
test_preds = log_model.predict(x_test)
print("Training Accuracy: %f" % accuracy_score(train_preds, y_train))
print("Training F1 Score: %f" % f1_score(train_preds, y_train))
print("Testing Accuracy: %f" % accuracy_score(test_preds, y_test))
print("Testing F1 Score: %f" % f1_score(test_preds, y_test))

# Decision Tree

In [ ]:
dtree_model = DecisionTreeClassifier()
dtree_model.fit(x_train, y_train)
train_preds = dtree_model.predict(x_train)
test_preds = dtree_model.predict(x_test)
print("Training Accuracy: %f" % accuracy_score(train_preds, y_train))
print("Training F1 Score: %f" % f1_score(train_preds, y_train))
print("Testing Accuracy: %f" % accuracy_score(test_preds, y_test))
print("Testing F1 Score: %f" % f1_score(test_preds, y_test))

# Random Forest

In [ ]:
forest_model = RandomForestClassifier()
forest_model.fit(x_train, y_train)
train_preds = forest_model.predict(x_train)
test_preds = forest_model.predict(x_test)
print("Training Accuracy: %f" % accuracy_score(train_preds, y_train))
print("Training F1 Score: %f" % f1_score(train_preds, y_train))
print("Testing Accuracy: %f" % accuracy_score(test_preds, y_test))
print("Testing F1 Score: %f" % f1_score(test_preds, y_test))

# AdaBoost

In [ ]:
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(x_train, y_train)
train_preds = adaboost_model.predict(x_train)
test_preds = adaboost_model.predict(x_test)
print("Training Accuracy: %f" % accuracy_score(train_preds, y_train))
print("Training F1 Score: %f" % f1_score(train_preds, y_train))
print("Testing Accuracy: %f" % accuracy_score(test_preds, y_test))
print("Testing F1 Score: %f" % f1_score(test_preds, y_test))

# Support Vector Machine

In [ ]:
svc_model = SVC()
svc_model.fit(x_train, y_train)
train_preds = svc_model.predict(x_train)
test_preds = svc_model.predict(x_test)
print("Training Accuracy: %f" % accuracy_score(train_preds, y_train))
print("Training F1 Score: %f" % f1_score(train_preds, y_train))
print("Testing Accuracy: %f" % accuracy_score(test_preds, y_test))
print("Testing F1 Score: %f" % f1_score(test_preds, y_test))

# Grid Search with Cross Validation

In [ ]:
adaboost_cv = GridSearchCV(AdaBoostClassifier(),
                          param_grid={"n_estimators":range(30, 101, 10),
                                     "learning_rate":[1, 0.1, 0.01]})
adaboost_cv.fit(x_train, y_train)
best_model = adaboost_cv.best_estimator_
best_model

In [ ]:
train_preds = best_model.predict(x_train)
test_preds = best_model.predict(x_test)
print("Training Accuracy: %f" % accuracy_score(train_preds, y_train))
print("Training F1 Score: %f" % f1_score(train_preds, y_train))
print("Testing Accuracy: %f" % accuracy_score(test_preds, y_test))
print("Testing F1 Score: %f" % f1_score(test_preds, y_test))

## Load epoched EEG data for Neural Net training

In [ ]:
train_df = pickle.load(open("data/epoched_train.pkl", "rb"))
train_df.head()

In [ ]:
train_df.drop(["patient_id", "start_time", "event_type"], axis=1).iloc[0].apply(lambda x:x.shape)

# Neural Networks
Looking at the raw data's shapes above, we can concatenate the 6 channels to create an input of 6000 values into a basic neural network

In [ ]:
X = train_df.drop(["patient_id", "start_time", "event_type"], axis=1).apply(lambda x:np.concatenate(x), axis=1)
X = np.array(X.values.tolist())
Y = train_df["event_type"].values.astype(float)
X.shape, Y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)
x_train.shape, x_test.shape

In [ ]:
neural_network = keras.Sequential([Dense(2048, activation="relu"),
                                   Dense(1024, activation="relu"),
                                   Dense(512, activation="relu"),
                                   Dense(128, activation="relu"),
                                   Dense(1, activation="sigmoid")])
opt = Adam(lr=0.001)
neural_network.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = neural_network.fit(x_train, y_train, epochs=15, batch_size=64)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 4))
axs[0].plot(history.history["loss"])
axs[0].set_title("Neural Network Training Loss")
axs[0].set_ylabel("Categorial Crossentropy")
axs[0].set_xlabel("Epoch")
axs[1].plot(history.history["accuracy"])
axs[1].set_title("Neural Network Training Accuracy")
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Epoch");

In [ ]:
neural_network.evaluate(x_test, y_test)

# Convolutional Neural Networks
To use a convolutional neural network architecture, we will instead stack the 1000 dimensional arrays to make 1000 x 6 matrices that can the model can perform convolution on.

In [ ]:
X = train_df.drop(["patient_id", "start_time", "event_type"], axis=1).apply(lambda x:np.stack(x, axis=-1), axis=1)
X = np.array(X.values.tolist())
X = X.reshape(list(X.shape)+[1])
Y = train_df["event_type"].values.astype(float)
X.shape, Y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)
x_train.shape, x_test.shape

In [ ]:
cnn = keras.Sequential([Conv2D(32, (3, 3), activation="relu", input_shape=(1000, 6, 1)),
                        Conv2D(64, (3, 3), activation="relu"),
                        Flatten(),
                        Dense(256, activation="relu"),
                        Dropout(0.2),
                        Dense(128, activation="relu"),
                        Dense(1, activation="sigmoid")])
opt = Adam(lr=0.001)
cnn.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = cnn.fit(x_train, y_train, epochs=8, batch_size=64)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 4))
axs[0].plot(history.history["loss"])
axs[0].set_title("CNN Training Loss")
axs[0].set_ylabel("Categorial Crossentropy")
axs[0].set_xlabel("Epoch")
axs[1].plot(history.history["accuracy"])
axs[1].set_title("CNN Training Accuracy")
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Epoch");

In [ ]:
cnn.evaluate(x_test, y_test)